# Esimerkki VQE algoritmista tequilalla

https://albacl.github.io/files/Tequila_2020.pdf

## Algoritmin tiedot

- Nimi: Tequila VQE example
- Algoritmin tyyppi: VQE(BFGS optimizer, UCCSD ansatz)
- Molekyyli: LiH (Lithium hydride)
 
## Metriikat

- iterations: 2
- measurements: ?
- circuit_depth: ?
- accuracy: algoritmin tulos on -7.979466, mikä on oikea arvo?

In [1]:
import tequila as tq

## Define Molecule

In [2]:
active = {'a1': [1], 'b1':[0]}
molecule = tq.quantumchemistry.Molecule(geometry='H 0.0 0.0 0.0\nLi 0.0 0.0 1.6', basis_set='6-31g', active_orbitals=active, transformation='bravyi-kitaev')
print(molecule)

<class 'tequila.quantumchemistry.psi4_interface.QuantumChemistryPsi4'>
Qubit Encoding
transformation=<function bravyi_kitaev at 0x7f2064325680>

Parameters
basis_set       : 6-31g           
geometry        : H 0.0 0.0 0.0
Li 0.0 0.0 1.6 
description     :                 
multiplicity    : 1               
charge          : 0               
closed_shell    : True            
name            : molecule        

Psi4 Data
Point Group (full) : c_inf_v         
Point Group (used) : c2v             
nirrep          : 4 
irreps          : ['A1', 'A2', 'B1', 'B2'] 
mos per irrep   : [7, 0, 2, 2]    
Active Space Data:
active_orbitals : [1, 4]          
reference_orbitals : [0, 1]          
frozen_docc     : [1, 0, 0, 0]    
frozen_uocc     : [5, 0, 1, 2]    

Orbitals:
0 : 0A1 energy = -2.453655 
1 : 1A1 energy = -0.300437 
2 : 2A1 energy = +0.009271 
3 : 0B2 energy = +0.060181 
4 : 0B1 energy = +0.060181 
5 : 3A1 energy = +0.145005 
6 : 4A1 energy = +0.200485 
7 : 1B1 energy = +0.221832 
8 

## Get the qubit hamiltonian

In [3]:
H = molecule.make_hamiltonian()
print(H)

-7.6325+0.1807Z(0)+0.1807Z(0)Z(1)+0.0141Z(2)+0.0141Z(1)Z(2)Z(3)+0.1185Z(1)+0.0025Y(0)Z(1)Y(2)Z(3)+0.0025X(0)Z(1)X(2)+0.0025X(0)Z(1)X(2)Z(3)+0.0025Y(0)Z(1)Y(2)+0.0428Z(0)Z(2)+0.0453Z(0)Z(1)Z(2)Z(3)+0.0453Z(0)Z(1)Z(2)+0.0428Z(0)Z(2)Z(3)+0.0439Z(1)Z(3)


## Make the UCCSD ansatz with cc2 ordering

In [4]:
U = molecule.make_uccsd_ansatz(initial_amplitudes='cc2', trotter_steps=1)
print(U)

circuit: 
X(target=(0,))
FermionicEx(target=(0, 1, 2, 3), control=(), parameter=Objective with 0 unique expectation values
variables = [(1, 0, 1, 0)]
types     = [])
FermionicEx(target=(0, 1, 2, 3), control=(), parameter=Objective with 0 unique expectation values
variables = [(1, 0, 1, 0)]
types     = [])



## Define the expectationvalue

In [5]:
E = tq.ExpectationValue(H=H, U=U)
print(E)

Objective with 1 unique expectation values
variables = [(1, 0, 1, 0)]
types     = not compiled


## Compute reference energies

In [6]:
fci = molecule.compute_energy('fci')
cisd = molecule.compute_energy('detci', options={'detci__ex_level': 2})
print(fci)
print(cisd)

-7.979466028301438
-7.979466028301438


## Optimize

In [7]:
initial_values = {k:0.0 for k in E.extract_variables()}
result = tq.minimize(objective=E, method='BFGS', gradient='2-point', method_options={'eps':1.e-3}, initial_values=initial_values)

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
device          : None
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
grad instr      : 2-point
gradient        : scipy numerical 2-point
hessian         : scipy numerical None

active variables : 1

E=-7.97932157  angles= {(1, 0, 1, 0): 0.0}  samples= None
E=-7.97932156  angles= {(1, 0, 1, 0): 1.4901161193847656e-08}  samples= None
E=-7.97945002  angles= {(1, 0, 1, 0): -0.019623160362243652}  samples= None
E=-7.97945002  angles= {(1, 0, 1, 0): -0.019623175263404846}  samples= None
E=-7.97946603  angles= {(1, 0, 1, 0): -0.014722356398789873}  samples= None
E=-7.97946603  angles= {(1, 0, 1, 0): -0.014722371299951067}  samples= None
Optimization terminated successfully.
         Current function value: -7.979466
         Iterations: 2
         Function evaluations: 6
         Gradient evaluations: 3


In [8]:
print('VQE : {:+2.8}'.format(result.energy))
print('CISD : {:+2.8}'.format(cisd))
print('FCI : {:+2.8}'.format(fci))

VQE : -7.979466
CISD : -7.979466
FCI : -7.979466


## Metrics

In [9]:
print(result)

SciPyResults(energy=-7.979466028301399, history=OptimizerHistory(energies=[-7.979450021620234, -7.979466028301394], gradients=[], angles=[{(1, 0, 1, 0): -0.019623175263404846}, {(1, 0, 1, 0): -0.014722371299951067}], energies_calls=[], gradients_calls=[], angles_calls=[]), variables=(1, 0, 1, 0) : -0.014722356398789873
, scipy_result=      fun: -7.979466028301399
 hess_inv: array([[0.75025409]])
      jac: array([-3.57627869e-07])
  message: 'Optimization terminated successfully.'
     nfev: 6
      nit: 2
     njev: 3
   status: 0
  success: True
        x: array([-0.01472236]))
